In [49]:
import os

In [50]:
model_parent_path = os.getcwd()
model_path = 'google-t5/new_t5-3b_qasc_epoch_2__eval_qasc_acc_72.57.pkl'
model_name = None
tokenizer_name = "google-t5/t5-3b"
max_answer_length = 300
BATCH_SIZE = 10
accumulate_step = None
DATA_NAME_SINGLE = "qasc"
NUM_EPOCHS = 1
VISIBLE_DEVICE = "0"
DEVICE = 0
TO_TEST=True

In [51]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_DEVICE)  # cannot work
print(
    f"Loi params: {model_path}{'#'*5}{max_answer_length}{'#'*5}batch_size={BATCH_SIZE};accumulate_step={accumulate_step}"
)
DATABASE_NAME = [DATA_NAME_SINGLE]  # ,'arc_hard']#,'race','mctest',]

Loi params: google-t5/new_t5-3b_qasc_epoch_2__eval_qasc_acc_72.57.pkl#####300#####batch_size=10;accumulate_step=None


In [52]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm.notebook import trange, tqdm
from random_utils import set_seed
import pandas as pd

In [53]:
# model_name = "loi_with_padding_just_same_answer_len_70.pkl"

# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)

model_original = (
    T5ForConditionalGeneration.from_pretrained(
        f"{model_parent_path}/{model_path}", device_map="auto"
    )
    if model_path
    else T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
)
model = model_original
# model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
dataset_test = []
[dataset_test.append([]) for x in DATABASE_NAME]
moi_index = []
# TO_TEST=False
for i, dataname in enumerate(DATABASE_NAME):
    dataset_test[i] = pickle.load(
        open(f"multiple_choice_datasets/{dataname}_{'test' if TO_TEST==True else 'validate'}.pkl", "rb")
    )
# TO_TEST=True

In [68]:
import textwrap
# dataset_test[0][0]
# ques=4
dataset_test[0][0][1]
# print('correct: ',dataset_test[ques][1],'length',len(dataset_test[0][0]))

'What type of birth do therian mammals have? \\n ( ) Anthax ( ) under water ( ) uterus ( ) wombs ( ) two ( ) moles ( ) live ( ) embryo'

In [91]:
import traceback
def hook(hook_before, oldfunc, hook_after):

    def foo(*args, **kwargs):
        hook_before(*args, **kwargs)
        aa = oldfunc(*args, **kwargs)
        hook_after(*args, **kwargs)
        return aa

    return foo
def new_compute_bias(self, query_length, key_length, device=None):
    """Compute binned relative position bias"""
    if device is None:
        device = self.relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[
        :, None
    ]
    memory_position = torch.arange(key_length, dtype=torch.long, device=device)[None, :]

    relative_position = (
        memory_position - context_position
    )  # shape (query_length, key_length)
    if self.is_decoder:
        values = values.permute([2, 0, 1]).unsqueeze(
            0
        )  # shape (1, num_heads, query_length, key_length)
        return values

    anchors = self.anchor_array
    values = []
    for anchor in anchors:
        mot = [[anchor[idx], anchor[idx + 1]] for idx in range(len(anchor) - 1)]
        if len(mot)>0:
            max_answer_length = max([x[1] - x[0] for x in mot])
            # print(a, b, c, d, max_answer_length)
            context_position_new = context_position.clone()
            for i in range(1, len(mot)):
                context_position_new[mot[i][0] : mot[i][1]] = (
                    context_position_new[mot[i][0] : mot[i][1]] - mot[0][0]
                )
            context_position_new[-1] = mot[0][0] + 2 * max_answer_length
            memory_position_new = context_position_new.clone().view(1, -1)
            relative_position = (
                memory_position_new - context_position_new
            )  # shape (query_length, key_length)
            for i in range(len(mot)):
                for j in range(len(mot)):
                    if i != j:
                        x = mot[i]
                        y = mot[j]
                        relative_position[x[0] : x[1], y[0] : y[1]] += max_answer_length
        relative_position_bucket = self._relative_position_bucket(
            relative_position,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )
        value = self.relative_attention_bias(relative_position_bucket)
        values.append(value)
    values = torch.stack(values)  # shape [1, 91, 91, 16]
    values = values.permute(
        [0, 3, 1, 2]
    )  # shape (batch size, num_heads, query_length, key_length)
    return values
import textwrap


extra_dim_learning = []


def set_mode(MODE):
    itself = model.encoder.block[0].layer[0].SelfAttention
    if MODE == "new":
        itself.compute_bias = partial(new_compute_bias, itself)
        model.forward = hook(
            input_before_hooker,
            partial(T5ForConditionalGeneration.forward, model),
            input_after_hooker,
        )

    else:
        itself.compute_bias = partial(
            model.encoder.block[0].layer[0].SelfAttention.__class__.compute_bias, itself
        )
        model.forward = partial(T5ForConditionalGeneration.forward, model)


def check_encoded(all_input_ids):
    anchors = []
    for input_ids in all_input_ids:
        # print('\n'.join([f'{x.item()},{y}' for x,y in zip(input_ids, tokens)][50:]))
        original = input_ids.tolist()
        anchor = []
        for i in range(len(input_ids)):
            if (
                i < len(input_ids) - 2
                and input_ids[i] == 41
                and input_ids[i + 1] == 3
                and input_ids[i + 2] == 61
            ) or original[i] == 1:
                anchor.append(i)
        anchors.append(anchor)
    return anchors


def input_before_hooker(*args, **kwargs):
    input_ids = kwargs["input_ids"]
    # print('old ',input_ids)
    anchors = check_encoded(input_ids)
    final_inputs = []
    for input_id, anchor in zip(input_ids, anchors):
        input_id = input_id.tolist()
        if len(anchor)>1:
            real_max_len = max(
                [anchor[idx + 1] - anchor[idx] for idx in range(len(anchor) - 1)]
            )
            if real_max_len > max_answer_length:
                print(f"ALERT: MAX LENGTH IS {real_max_len}")
            for x in reversed(range(1, len(anchor))):
                if anchor[x] - anchor[x - 1] < max_answer_length:
                    [
                        input_id.insert(anchor[x], 0)
                        for _ in range(max_answer_length - (anchor[x] - anchor[x - 1]))
                    ]

        final_inputs.append(input_id)

    max_length = max([len(input) for input in final_inputs])
    mask = [[1] * max_length] * len(final_inputs)
    for idx, input in enumerate(final_inputs):
        for x in range(max_length):
            if x >= len(input):
                mask[idx][x] = 0
        for x in range(max_length - len(input)):
            input.append(0)
    kwargs["input_ids"] = torch.tensor(final_inputs).to(input_ids.device)
    kwargs["attention_mask"] = torch.tensor(mask).to(input_ids.device)
    print('new ',kwargs["input_ids"])
    print('attention_mask ',kwargs["attention_mask"])
    traceback.print_stack()
    anchors = check_encoded(kwargs["input_ids"])
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = anchors


def input_after_hooker(*args, **kwargs):
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = None

def measure_unalike(arr, print_arr=False):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    if print_arr:
        print(arr)
    return 1 - ((arr / n) ** 2).sum()

def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(max_answer_length):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[: y + 1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(input_strs, padding=True, return_tensors="pt")

    answer = get_model_forward(
        input_ids_wrapper["input_ids"], input_ids_wrapper["attention_mask"]
    )
    return answer
# set_mode('old')
set_mode("new")

In [93]:

run_model('quna nham ')


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    self._run_once()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once
    handle._run()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/events.py", line 80, in _run
  

new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    self._run_once()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once
    handle._run()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/events.py", line 80, in _run
  

new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
new  tensor([[ 546,   29,    9,    3,   29, 1483,    1]], device='cuda:0')
attention_mask  tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/work/cvp352/loi_work/env/loi/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    self._run_once()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once
    handle._run()
  File "/home/cvp352/miniconda3/lib/python3.11/asyncio/events.py", line 80, in _run
  

['quna nham']

In [60]:
kk = [
    (index, x, y)
    for index, (x, y) in enumerate(model.named_parameters())
    if y.requires_grad == True
]
[(index, x) for index, x, y in kk if "decoder" in x]
len(kk)
all_position_weight = [
    y
    for index, x, y in kk
    if ("extra_dimension_embedding" in x)
    or (("encoder" in x) and ("relative_attention_bias" in x))
]
to_train_model = [y for index, x, y in kk]

In [61]:
def getAnswer(numeral):
    return chr(ord('A')+numeral)

In [62]:
from difflib import SequenceMatcher
def similar(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()


In [63]:
import csv
def evaluate_test():
    data = dataset_test[0]
    mal=0
    bad_choices=[]
    with open(f"{model_path.replace('/','_')}_{DATA_NAME_SINGLE}_test_submission.csv", 'w', newline='') as f:
        pbar=trange(len(data))
        for ques in pbar:
            id, question = data[ques]
            # question = data[ques][0]
            answer = run_model(question)[0]
            choices=question.split(' ( ) ')[1:]
            if answer in choices: 
                numeral=choices.index(answer)
                letter=getAnswer(numeral)
            else:
                mal+=1
                bad_choices.append((ques,answer))
                selection=np.argmax([similar(answer,x) for x in choices])
                letter=getAnswer(selection)
            chuoi=f'"{id}","{letter}"\n'
            f.write(chuoi)
            pbar.set_postfix_str(f'{mal}/{ques+1}={mal/(ques+1)*100:0.02f}%')
    return bad_choices 
if TO_TEST:
    bad_choices=evaluate_test()

  0%|          | 0/920 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import numpy as np
for x in bad_choices:
    ques=dataset_test[0][x[0]][1]
    choices=ques.split(' ( ) ')[1:]
    ans=x[1]
    print(x[0], choices, 'Fabricated answer ',ans)
    print('='*50)

In [ ]:

def evaluate_validate():
    ll = 1
    wrong_answers = [[] for x in range(ll)]
    got_2 = [[] for x in range(ll)]
    got_1 = [[] for x in range(ll)]
    answers = [[] for x in range(ll)]
    last_str = None
    last_acc = None
    for ix in range(ll):
        print(f"Name {DATABASE_NAME[ix]}")
        # [pickle.load(open(f"multiple_choice_datasets/obqa_fact_test.pkl", "rb"))]:
        # print(f"test {data==dataset_test}")
        count = 0
        count1 = 0
        count2 = 0
        count10 = 0
        total = 0
        data = dataset_test[ix]
        pbar1 = trange(len(data))
        for ques in pbar1:
            question = data[ques][0]
            key = data[ques][1]
            total += 1
            answer = run_model(question)[0]
            answers[ix].append(answer)
            if key == answer:
                count += 1
            else:
                wrong_answers[ix].append(ques)
            if key[0] == answer[0]:
                count1 += 1
                got_1[ix].append(ques)
            if key[:2] == answer[:2]:
                count2 += 1
                got_2[ix].append(ques)
            if answer in question:
                count10 += 1
            last_str = f"{count}, {count1}, {count2}, {count10},{total},{count/total*100:.2f},{count10/total*100:.2f}"
            last_acc = f"{count/total*100:.2f}"
            pbar1.set_postfix_str(last_str)
    return last_str, last_acc,wrong_answers

In [ ]:
if not TO_TEST:
    last_str, last_acc,wrong_answers= evaluate()
    print(last_str, last_acc,wrong_answers)

### Measure resilient

In [ ]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n) ** 2).sum()


measure_unalike(["a", "a", "a"])